In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import regex as re
import pgeocode

In [2]:
df = pd.read_csv("../data/Geospatial_Coordinates.csv")

In [3]:
df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [4]:
import wikipedia as wp

In [5]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df_wiki = pd.read_html(html)[0]

In [6]:
df_wiki.head()

,0,1,2,3,4,5,6,7,8
0,M1A Not assigned,M2A Not assigned,M3A North York (Parkwoods),M4A North York (Victoria Village),M5A Downtown Toronto (Regent Park / Harbourfront),M6A North York (Lawrence Manor / Lawrence Heig...,M7A Queen's Park (Ontario Provincial Government),M8A Not assigned,M9A Etobicoke (Islington Avenue)
1,M1B Scarborough (Malvern / Rouge),M2B Not assigned,M3B North York (Don Mills) North,M4B East York (Parkview Hill / Woodbine Gardens),"M5B Downtown Toronto (Garden District, Ryerson)",M6B North York (Glencairn),M7B Not assigned,M8B Not assigned,M9B Etobicoke (West Deane Park / Princess Gard...
2,M1C Scarborough (Rouge Hill / Port Union / Hig...,M2C Not assigned,M3C North York (Don Mills) South (Flemingdon P...,M4C East York (Woodbine Heights),M5C Downtown Toronto (St. James Town),M6C York (Humewood-Cedarvale),M7C Not assigned,M8C Not assigned,M9C Etobicoke (Eringate / Bloordale Gardens / ...
3,M1E Scarborough (Guildwood / Morningside / Wes...,M2E Not assigned,M3E Not assigned,M4E East Toronto (The Beaches),M5E Downtown Toronto (Berczy Park),M6E York (Caledonia-Fairbanks),M7E Not assigned,M8E Not assigned,M9E Not assigned
4,M1G Scarborough (Woburn),M2G Not assigned,M3G Not assigned,M4G East York (Leaside),M5G Downtown Toronto (Central Bay Street),M6G Downtown Toronto (Christie),M7G Not assigned,M8G Not assigned,M9G Not assigned


In [7]:
df_wiki.shape

(20, 9)

In [8]:
neighbourhoods=[]
def process_data(df):
    for index,row in df.iterrows():
        for col in df.columns:
            neighbourhoods.append([
                                     row[col][:3],
                                     row[col][row[col].find("(")+1:row[col].find(")")]
                                    ])

In [9]:
process_data(df_wiki)

In [10]:
df_hoods = pd.DataFrame(neighbourhoods, columns=['Postal Code','neighbourhood'])

In [11]:
df_neighbourhoods = df.merge(df_hoods, on='Postal Code', how='left')

In [12]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood
0,M1B,43.806686,-79.194353,Malvern / Rouge
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill
3,M1G,43.770992,-79.216917,Woburn
4,M1H,43.773136,-79.239476,Cedarbrae


In [13]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood
0,M1B,43.806686,-79.194353,Malvern / Rouge
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill
3,M1G,43.770992,-79.216917,Woburn
4,M1H,43.773136,-79.239476,Cedarbrae


In [14]:
df_neighbourhoods['hood_splits'] = df_neighbourhoods['neighbourhood'].apply(lambda x: x.split(' / '))

In [15]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]"
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]"
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]"
3,M1G,43.770992,-79.216917,Woburn,[Woburn]
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae]


In [16]:
df_neighbourhoods['num_hood_splits'] = df_neighbourhoods['neighbourhood'].apply(lambda x: len(x.split(' / ')))
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1


In [17]:
def hood_split(hoods):
    n_splits = len(hoods.split(' / '))
    max_len = df_neighbourhoods.num_hood_splits.max()
    num_pads = max_len - n_splits
    lst = hoods.split(' / ')
    lst += [np.nan]*num_pads
    return lst

In [18]:
df_neighbourhoods["hood_splits_padded"] = df_neighbourhoods["neighbourhood"].apply(lambda x: hood_split(x))

In [19]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits,hood_splits_padded
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2,"[Malvern, Rouge, nan, nan, nan, nan, nan, nan,..."
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3,"[Rouge Hill, Port Union, Highland Creek, nan, ..."
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3,"[Guildwood, Morningside, West Hill, nan, nan, ..."
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1,"[Woburn, nan, nan, nan, nan, nan, nan, nan, nan]"
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1,"[Cedarbrae, nan, nan, nan, nan, nan, nan, nan,..."


In [20]:
def sep_hood_columns(df):
    for i in range(len(df.hood_splits_padded[0])):
        df[f"hood_{i}"] = df['hood_splits_padded'].apply(lambda x: x[i])
    return df

In [21]:
sep_hood_columns(df_neighbourhoods).head()

,Postal Code,Latitude,Longitude,neighbourhood,hood_splits,num_hood_splits,hood_splits_padded,hood_0,hood_1,hood_2,hood_3,hood_4,hood_5,hood_6,hood_7,hood_8
0,M1B,43.806686,-79.194353,Malvern / Rouge,"[Malvern, Rouge]",2,"[Malvern, Rouge, nan, nan, nan, nan, nan, nan,...",Malvern,Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,43.784535,-79.160497,Rouge Hill / Port Union / Highland Creek,"[Rouge Hill, Port Union, Highland Creek]",3,"[Rouge Hill, Port Union, Highland Creek, nan, ...",Rouge Hill,Port Union,Highland Creek,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,43.763573,-79.188711,Guildwood / Morningside / West Hill,"[Guildwood, Morningside, West Hill]",3,"[Guildwood, Morningside, West Hill, nan, nan, ...",Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,43.770992,-79.216917,Woburn,[Woburn],1,"[Woburn, nan, nan, nan, nan, nan, nan, nan, nan]",Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,43.773136,-79.239476,Cedarbrae,[Cedarbrae],1,"[Cedarbrae, nan, nan, nan, nan, nan, nan, nan,...",Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_neighbourhoods.drop(['neighbourhood','hood_splits','num_hood_splits','hood_splits_padded'], axis=1, inplace=True)

In [23]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,hood_0,hood_1,hood_2,hood_3,hood_4,hood_5,hood_6,hood_7,hood_8
0,M1B,43.806686,-79.194353,Malvern,Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,43.784535,-79.160497,Rouge Hill,Port Union,Highland Creek,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,43.763573,-79.188711,Guildwood,Morningside,West Hill,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,43.770992,-79.216917,Woburn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,43.773136,-79.239476,Cedarbrae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
new_hoods = []
def add_hoods(df):
    df_new = df.iloc[:,4:]
    for index,row in df_new.iterrows():
        for col in df_new.columns:
            if pd.isnull(row[col]) == False:
                new_hoods.append([
                                    df['Postal Code'][index],
                                    df['Latitude'][index],
                                    df['Longitude'][index],
                                    df_new.loc[index,col]
                                 ])
    return          

In [25]:
add_hoods(df_neighbourhoods)

In [26]:
df_neighbourhoods.drop(df_neighbourhoods.columns[4:], axis=1, inplace=True)

In [27]:
df_neighbourhoods.head()

,Postal Code,Latitude,Longitude,hood_0
0,M1B,43.806686,-79.194353,Malvern
1,M1C,43.784535,-79.160497,Rouge Hill
2,M1E,43.763573,-79.188711,Guildwood
3,M1G,43.770992,-79.216917,Woburn
4,M1H,43.773136,-79.239476,Cedarbrae


In [28]:
df_new_hoods = pd.DataFrame(new_hoods, columns = df_neighbourhoods.columns)

In [29]:
df_new_hoods

,Postal Code,Latitude,Longitude,hood_0
0,M1B,43.806686,-79.194353,Rouge
1,M1C,43.784535,-79.160497,Port Union
2,M1C,43.784535,-79.160497,Highland Creek
3,M1E,43.763573,-79.188711,Morningside
4,M1E,43.763573,-79.188711,West Hill
...,...,...,...,...
107,M9W,43.706748,-79.594054,Kipling Heights
108,M9W,43.706748,-79.594054,Rexdale
109,M9W,43.706748,-79.594054,Elms
110,M9W,43.706748,-79.594054,Tandridge


In [30]:
df_all_hoods = pd.concat([df_neighbourhoods, df_new_hoods])

In [31]:
df_all_hoods.rename(columns={"hood_0": "HoodName"}, inplace=True)